# Scene Graph to Text Summarization

This notebook demonstrates how to convert scene graphs into natural language descriptions and generate summaries using various text summarization models.

## Overview

1. **Scene Graph Processing**: Load and process scene graph data
2. **Text Generation**: Convert scene graph triples to natural language
3. **Summarization**: Generate concise summaries using different models
4. **Evaluation**: Compare different summarization approaches

## Prerequisites

Make sure you have the required dependencies installed:
```bash
pip install torch transformers sentencepiece protobuf
```


In [ ]:
# Import required libraries
import sys
import os
import json
import torch
import numpy as np
from pathlib import Path
from typing import List, Dict, Any, Tuple

# Add the src directory to the path
sys.path.append(str(Path.cwd().parent / "src"))

# Import m3sgg summarization components
from m3sgg.language.summarization.summarize import (
    linearize_triples,
    summarize_sentences,
    summarize_with_pegasus_separate,
    summarize_with_pegasus_custom
)
from m3sgg.language.summarization.wrappers import (
    T5SummarizationWrapper,
    PegasusSummarizationWrapper,
    PegasusCustomConfig,
    PegasusSeparateLoader
)

print("Libraries imported successfully!")


## 1. Sample Scene Graph Data

Let's start with some sample scene graph data to demonstrate the summarization process.


In [ ]:
# Sample scene graph triples
sample_triples = [
    ("person", "holding", "cup"),
    ("person", "sitting on", "chair"),
    ("person", "looking at", "laptop"),
    ("laptop", "on", "table"),
    ("cup", "on", "table"),
    ("person", "wearing", "glasses"),
    ("person", "touching", "keyboard"),
    ("laptop", "in front of", "person"),
    ("chair", "behind", "table"),
    ("person", "drinking from", "cup")
]

print("Sample scene graph triples:")
for i, triple in enumerate(sample_triples, 1):
    subject, predicate, obj = triple
    print(f"{i:2d}. {subject} --{predicate}--> {obj}")

print(f"\nTotal triples: {len(sample_triples)}")


## 2. Convert Triples to Natural Language

Convert the scene graph triples into natural language sentences using the linearization function.


In [ ]:
# Convert triples to natural language sentences
sentences = linearize_triples(sample_triples)

print("Natural language sentences:")
print("=" * 50)
for i, sentence in enumerate(sentences, 1):
    print(f"{i:2d}. {sentence}")

print(f"\nTotal sentences: {len(sentences)}")

# Combine all sentences into a single text
combined_text = " ".join(sentences)
print(f"\nCombined text length: {len(combined_text)} characters")
print(f"Combined text: {combined_text}")


## 3. Text Summarization with Different Models

Now let's generate summaries using different summarization models and compare their outputs.


In [ ]:
# Function to safely run summarization with error handling
def safe_summarize(sentences, model_type="t5", model_name=None, **kwargs):
    """Safely run summarization with error handling.
    
    :param sentences: List of sentences to summarize
    :param model_type: Type of model to use ('t5' or 'pegasus')
    :param model_name: Specific model name to use
    :param kwargs: Additional arguments for summarization
    :return: Summary text or error message
    """
    try:
        if model_type.lower() == "t5":
            model_name = model_name or "google-t5/t5-base"
            summary = summarize_sentences(sentences, model_name=model_name, model_type="t5")
        elif model_type.lower() == "pegasus":
            model_name = model_name or "google/pegasus-xsum"
            summary = summarize_sentences(sentences, model_name=model_name, model_type="pegasus")
        else:
            return f"Unsupported model type: {model_type}"
        
        return summary
    except Exception as e:
        return f"Error with {model_type}: {str(e)}"

# Test different summarization approaches
print("Testing different summarization models...")
print("=" * 60)

# T5 Summarization
print("T5 Summarization:")
print("-" * 30)
t5_summary = safe_summarize(sentences, model_type="t5")
print(f"Summary: {t5_summary}")
print()

# Pegasus Summarization
print("Pegasus Summarization:")
print("-" * 30)
pegasus_summary = safe_summarize(sentences, model_type="pegasus")
print(f"Summary: {pegasus_summary}")
print()


## 4. Advanced Summarization Techniques

Let's explore more advanced summarization techniques with custom configurations.


In [ ]:
# Advanced Pegasus configurations
print("Advanced Pegasus Summarization:")
print("=" * 60)

# Pegasus with separate loader
print("Pegasus Separate Loader:")
print("-" * 30)
try:
    pegasus_separate_summary = summarize_with_pegasus_separate(sentences)
    print(f"Summary: {pegasus_separate_summary}")
except Exception as e:
    print(f"Error: {e}")
print()

# Pegasus with custom configuration
print("Pegasus Custom Configuration:")
print("-" * 30)
try:
    pegasus_custom_summary = summarize_with_pegasus_custom(
        sentences, 
        max_length=80, 
        min_length=15, 
        length_penalty=1.5, 
        num_beams=6
    )
    print(f"Summary: {pegasus_custom_summary}")
except Exception as e:
    print(f"Error: {e}")
print()


## 5. Batch Processing and Comparison

Let's process multiple scene graphs and compare the summarization results.


In [ ]:
# Multiple scene graph examples
scene_graphs = [
    {
        "name": "Office Scene",
        "triples": [
            ("person", "sitting on", "chair"),
            ("person", "looking at", "computer"),
            ("person", "typing on", "keyboard"),
            ("computer", "on", "desk"),
            ("person", "wearing", "glasses")
        ]
    },
    {
        "name": "Kitchen Scene", 
        "triples": [
            ("person", "standing in", "kitchen"),
            ("person", "holding", "knife"),
            ("person", "cutting", "vegetables"),
            ("vegetables", "on", "cutting board"),
            ("person", "wearing", "apron")
        ]
    },
    {
        "name": "Living Room Scene",
        "triples": [
            ("person", "sitting on", "sofa"),
            ("person", "watching", "television"),
            ("person", "holding", "remote"),
            ("television", "in front of", "sofa"),
            ("person", "drinking from", "coffee cup")
        ]
    }
]

# Process each scene graph
results = []

for scene in scene_graphs:
    print(f"Processing {scene['name']}...")
    print("-" * 40)
    
    # Convert to sentences
    scene_sentences = linearize_triples(scene['triples'])
    print("Original sentences:")
    for sentence in scene_sentences:
        print(f"  - {sentence}")
    
    # Generate summary
    summary = safe_summarize(scene_sentences, model_type="t5")
    print(f"Summary: {summary}")
    
    # Store results
    results.append({
        "name": scene['name'],
        "sentences": scene_sentences,
        "summary": summary,
        "num_triples": len(scene['triples'])
    })
    
    print()
    print("=" * 60)
    print()

print(f"Processed {len(results)} scene graphs successfully!")


## 6. Summary Analysis and Metrics

Let's analyze the summarization results and compute some basic metrics.


In [ ]:
# Analyze summarization results
def analyze_summaries(results):
    """Analyze summarization results and compute metrics.
    
    :param results: List of summarization results
    :return: Dictionary with analysis metrics
    """
    analysis = {
        "total_scenes": len(results),
        "avg_triples_per_scene": 0,
        "avg_sentences_per_scene": 0,
        "avg_summary_length": 0,
        "compression_ratios": [],
        "successful_summaries": 0
    }
    
    total_triples = 0
    total_sentences = 0
    total_summary_chars = 0
    
    for result in results:
        total_triples += result["num_triples"]
        total_sentences += len(result["sentences"])
        
        summary = result["summary"]
        if not summary.startswith("Error"):
            analysis["successful_summaries"] += 1
            total_summary_chars += len(summary)
            
            # Calculate compression ratio
            original_length = sum(len(s) for s in result["sentences"])
            compression_ratio = len(summary) / original_length if original_length > 0 else 0
            analysis["compression_ratios"].append(compression_ratio)
    
    analysis["avg_triples_per_scene"] = total_triples / len(results)
    analysis["avg_sentences_per_scene"] = total_sentences / len(results)
    analysis["avg_summary_length"] = total_summary_chars / max(analysis["successful_summaries"], 1)
    
    return analysis

# Perform analysis
analysis = analyze_summaries(results)

print("Summarization Analysis:")
print("=" * 50)
print(f"Total scenes processed: {analysis['total_scenes']}")
print(f"Successful summaries: {analysis['successful_summaries']}")
print(f"Average triples per scene: {analysis['avg_triples_per_scene']:.1f}")
print(f"Average sentences per scene: {analysis['avg_sentences_per_scene']:.1f}")
print(f"Average summary length: {analysis['avg_summary_length']:.1f} characters")

if analysis['compression_ratios']:
    avg_compression = sum(analysis['compression_ratios']) / len(analysis['compression_ratios'])
    print(f"Average compression ratio: {avg_compression:.2f}")
    print(f"Compression range: {min(analysis['compression_ratios']):.2f} - {max(analysis['compression_ratios']):.2f}")

print("\nDetailed Results:")
print("-" * 30)
for i, result in enumerate(results, 1):
    print(f"{i}. {result['name']}")
    print(f"   Triples: {result['num_triples']}, Sentences: {len(result['sentences'])}")
    print(f"   Summary: {result['summary'][:100]}{'...' if len(result['summary']) > 100 else ''}")
    print()


## 7. Export and Save Results

Finally, let's export the summarization results for further use.


In [ ]:
# Export results to JSON
def export_summarization_results(results, analysis, output_path="summarization_results.json"):
    """Export summarization results to JSON format.
    
    :param results: List of summarization results
    :param analysis: Analysis metrics
    :param output_path: Path to save the JSON file
    :return: Dictionary with exported data
    """
    export_data = {
        "metadata": {
            "timestamp": "2024-01-01T00:00:00",  # Would use datetime.now().isoformat() in real usage
            "total_scenes": analysis["total_scenes"],
            "successful_summaries": analysis["successful_summaries"],
            "avg_compression_ratio": sum(analysis["compression_ratios"]) / len(analysis["compression_ratios"]) if analysis["compression_ratios"] else 0
        },
        "analysis": analysis,
        "results": results
    }
    
    # Save to file
    with open(output_path, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"Results exported to {output_path}")
    return export_data

# Export results
export_data = export_summarization_results(results, analysis)
print(f"Exported {len(export_data['results'])} scene graph summarization results")

# Display summary
print("\nExport Summary:")
print("-" * 20)
print(f"File: summarization_results.json")
print(f"Scenes: {export_data['metadata']['total_scenes']}")
print(f"Successful: {export_data['metadata']['successful_summaries']}")
print(f"Avg compression: {export_data['metadata']['avg_compression_ratio']:.2f}")


## Summary

This notebook demonstrated the complete pipeline for converting scene graphs to text summaries:

1. **Scene Graph Processing**: Loaded and processed scene graph triples
2. **Text Generation**: Converted triples to natural language using the `linearize_triples` function
3. **Summarization**: Applied different summarization models (T5, Pegasus) with various configurations
4. **Batch Processing**: Processed multiple scene graphs and compared results
5. **Analysis**: Computed metrics like compression ratios and success rates
6. **Export**: Saved results for further analysis

### Key Features Demonstrated

- **Multiple Models**: T5 and Pegasus summarization models
- **Custom Configurations**: Advanced settings for different use cases
- **Error Handling**: Robust error handling for model loading and inference
- **Batch Processing**: Efficient processing of multiple scene graphs
- **Metrics and Analysis**: Comprehensive analysis of summarization quality

### Next Steps

- Try different model architectures (BART, GPT-based models)
- Experiment with different summarization prompts
- Integrate with video processing pipeline (see next notebook)
- Explore domain-specific summarization approaches
- Implement evaluation metrics (ROUGE, BLEU, etc.)

### Troubleshooting

If you encounter issues:

1. **Model Loading Errors**: Ensure transformers library is properly installed
2. **Memory Issues**: Use smaller models or reduce batch sizes
3. **CUDA Errors**: Check GPU availability and model compatibility
4. **Import Errors**: Verify the m3sgg package is properly installed
